In [1]:
def generate_problem(m, N_bits, p_bits, r_bits):
    p = random_prime(2^N_bits - 1, false, 2^(N_bits-1))
    q_bits = N_bits - p_bits
    q = randint(2^(q_bits - 1), 2^q_bits - 1)
    N = p * q # exact multiple of p
    a_lst = [] # list of approximate multiples of p 
    for i in range(m):
        q = randint(2^(q_bits - 1), 2^q_bits - 1)
        r = randint(2^(r_bits - 1), 2^r_bits - 1)
        a = p * q + r
        a_lst.append(a)
    return (N, a_lst)

In [2]:
m = 1
N_bits = 1000
p_bits = 200
r_bits = 36
problem_instance = generate_problem(m, N_bits, p_bits, r_bits)
(N, a_lst) = problem_instance

In [3]:
beta = 1
# hardcode these constants for now, ask about them later
t = 41
k = 8
X = 2^r_bits - 1 # the bound on the error

In [4]:
# iterate over i_1 + ... + i_m <= t
# do the "combo of dividing lines" approach
i_l_lst = []
from itertools import combinations
for combo in combinations(range(1, t + m + 1), m):
    i_combo = []
    all_dividing_lines = [0, *list(combo), t + m + 1]
    for j in range(1, len(all_dividing_lines) - 1):
        i_j = all_dividing_lines[j + 1] - all_dividing_lines[j] - 1
        i_combo.append(i_j)
    l = max(0, k - sum(i_combo))
    i_l_lst.append((i_combo, l))

In [5]:
len(i_l_lst)

42

In [6]:
R = PolynomialRing(QQ, names=["x" + str(i) for i in range(1, m + 1)], order="lex")
polys = []
polys_lite = []
for (i_combo, l) in i_l_lst:
    poly = N^l
    poly_lite = N^l
    for (x_j, a, i) in zip(R.gens(), a_lst, i_combo):
        poly *= (X * x_j - a)^i
        poly_lite *= (x_j - a)^i
    polys.append(poly)
    polys_lite.append(poly_lite)

In [7]:
def get_coeff_vector(poly, max_order_monomial):
    # credit to https://stackoverflow.com/questions/42812634/multivariate-polynomial-coefficients-including-zeros
    return [poly.monomial_coefficient(m) for m in monomial_all_divisors(max_order_monomial)] + [poly.constant_coefficient()]

def get_poly(coeff_vector, max_order_monomial):
    poly = 0
    all_monomials = monomial_all_divisors(max_order_monomial) + [1]
    for (m, coeff) in zip(coeff_vector, all_monomials):
        poly += coeff * m
    return poly

In [8]:
def monomial_all_divisors(monomial):
    if len(R.gens()) > 1:
        return R.monomial_all_divisors(monomial)
    else:
        max_i = list(monomial.dict().keys())[0]
        return [R.gens()[0]^i for i in range(1, max_i + 1)]

In [9]:
max_order_monomial = 1
for x_j in R.gens():
    max_order_monomial *= x_j^t
max_order_monomial

x1^41

In [10]:
monomial_all_divisors(max_order_monomial)

[x1,
 x1^2,
 x1^3,
 x1^4,
 x1^5,
 x1^6,
 x1^7,
 x1^8,
 x1^9,
 x1^10,
 x1^11,
 x1^12,
 x1^13,
 x1^14,
 x1^15,
 x1^16,
 x1^17,
 x1^18,
 x1^19,
 x1^20,
 x1^21,
 x1^22,
 x1^23,
 x1^24,
 x1^25,
 x1^26,
 x1^27,
 x1^28,
 x1^29,
 x1^30,
 x1^31,
 x1^32,
 x1^33,
 x1^34,
 x1^35,
 x1^36,
 x1^37,
 x1^38,
 x1^39,
 x1^40,
 x1^41]

In [11]:
coeff_matrix = matrix(ZZ, [get_coeff_vector(poly, max_order_monomial) for poly in polys])
coeff_matrix_lite = matrix(ZZ, [get_coeff_vector(poly, max_order_monomial) for poly in polys_lite])

In [12]:
coeff_matrix_lite.nrows()

42

In [13]:
(reduced_matrix, transformation_matrix) = coeff_matrix.LLL(transformation=True)

KeyboardInterrupt: 

In [630]:
# credit to https://doc.sagemath.org/html/en/constructions/polynomials.html#roots-of-multivariate-polynomials
reduced_matrix_lite = transformation_matrix * coeff_matrix_lite
reduced_polys_lite = [get_poly(v.list(), max_order_monomial) for v in reduced_matrix_lite[:m]]

In [631]:
reduced_polys_lite[0].roots()

[(41323006830, 7)]

In [632]:
r_1 = 41323006830
gcd(a_lst[0] - r_1, N)

5569496470052038351198963392157639997641575860060770720858522608519570329040554144842064525156707230247556419775092940080786461729235129864641191337499415861321828620189804354396250050969619085260728697870581076083500518617288863756155461136323225248370655255019001561965035301564367358011159651942183